# MNIST 다중 클래스 분류 (Keras 활용)

## 1. 들어가며

지난 시간까지 NumPy로 신경망의 핵심 원리(순전파, 역전파)를 직접 구현해보고, 이를 Keras와 PyTorch 프레임워크로도 구현해보면서 딥러닝의 가장 기본적인 작동 방식을 익혔습니다. AND 게이트라는 간단한 예제를 통해 신경망이 데이터를 학습하는 과정이었습니다.

하지만 현실 세계의 데이터는 AND 게이트처럼 단순하지 않고, 문제도 이진 분류(Binary Classification, 예: 합격/불합격)를 넘어 **다중 클래스 분류(Multi-class Classification, 예: 숫자 0~9 분류)**나 회귀(Regression, 예: 집값 예측) 등 다양한 형태를 가집니다.

이제 지금까지 배운 지식을 발판 삼아, 조금 더 실제적인 데이터셋을 가지고 다중 클래스 분류 문제를 해결하는 방법을 배워보겠습니다. 이번 실습에서는 **MNIST (Modified National Institute of Standards and Technology)** 데이터셋을 사용할 것입니다. MNIST는 0부터 9까지의 손글씨 이미지로 구성된 데이터셋으로, 딥러닝 분야에서 'Hello, World!'처럼 사용되는 유명한 데이터셋입니다.

이 노트북에서는 Keras를 사용하여 다음 내용들을 학습합니다.

1.  **MNIST 데이터셋 로딩 및 이해**
2.  **다중 클래스 분류를 위한 네트워크 구조 (출력층 활성화 함수: Softmax)**
3.  **다중 클래스 분류를 위한 손실 함수 (Cross-Entropy)**
4.  **새로운 옵티마이저 (Adam) 사용**
5.  **미니 배치 학습**
6.  **모델 평가 및 예측**
7.  **과적합(Overfitting) 방지를 위한 Dropout 소개**


In [ ]:
# 필요한 라이브러리를 미리 임포트합니다.
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.utils import to_categorical # One-Hot Encoding을 위해 사용
import matplotlib.pyplot as plt # 학습 과정 시각화를 위해 사용


## 2. MNIST 데이터셋 로딩 및 준비

MNIST 데이터셋은 Keras에 내장되어 있어 쉽게 불러올 수 있습니다. 이미 학습(train) 데이터와 테스트(test) 데이터로 나뉘어 있습니다.

### 2.1 데이터 로딩


In [ ]:
# MNIST 데이터셋 로딩
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()

print(f"학습 이미지 형태: {train_images.shape}") # (샘플 수, 높이, 너비)
print(f"학습 레이블 형태: {train_labels.shape}") # (샘플 수)
print(f"테스트 이미지 형태: {test_images.shape}")
print(f"테스트 레이블 형태: {test_labels.shape}")


*   `train_images`: 60,000개의 28x28 픽셀 이미지입니다. 각 픽셀 값은 0부터 255까지의 정수입니다.
*   `train_labels`: 60,000개의 이미지에 해당하는 레이블 (0부터 9까지의 숫자)입니다.
*   `test_images`: 10,000개의 테스트 이미지입니다.
*   `test_labels`: 10,000개의 테스트 레이블입니다.

### 2.2 데이터 시각화 (샘플 확인)

데이터가 어떻게 생겼는지 몇 개 살펴봅시다.


In [ ]:
# 첫 번째 이미지를 시각화합니다.
plt.figure(figsize=(5, 5))
plt.imshow(train_images[0], cmap='gray')
plt.title(f"Label: {train_labels[0]}")
plt.axis('off')
plt.show()

# 몇 개의 샘플 더 시각화
plt.figure(figsize=(10, 5))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(train_images[i], cmap='gray')
    plt.title(f"{train_labels[i]}")
    plt.axis('off')
plt.tight_layout()
plt.show()


### 2.3 데이터 전처리 (Preprocessing)

신경망에 데이터를 입력하기 전에 몇 가지 전처리 과정이 필요합니다.

1.  **픽셀 값 정규화 (Normalization):** 픽셀 값이 0~255 범위를 가지는데, 이를 0~1 사이의 값으로 정규화하면 학습이 더 잘 됩니다. 간단하게 255로 나누어 줍니다.
2.  **이미지 형태 변환 (Flattening):** 우리가 지금까지 다룬 신경망(Dense 레이어)은 입력이 1차원 벡터 형태여야 합니다. 28x28 이미지를 784 (28*28) 길이의 1차원 벡터로 펼쳐야 합니다.
3.  **레이블 One-Hot Encoding:** 현재 레이블은 숫자 0부터 9까지의 정수입니다. 다중 클래스 분류에서 크로스 엔트로피 손실 함수를 사용하기 위해서는 레이블을 **원-핫 인코딩(One-Hot Encoding)** 형태로 바꿔줘야 합니다. 예를 들어, 숫자 '3'은 `[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]` (인덱스 3만 1이고 나머지는 0인 10차원 벡터)로 표현됩니다.


In [ ]:
# 1. 픽셀 값 정규화: 0-255 값을 0-1 값으로 스케일링
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# 2. 이미지 형태 변환 (Flattening): 28x28 이미지를 784 길이의 1차원 벡터로 펼침
# Keras의 Flatten 레이어를 사용해도 되지만, 여기서는 직접 변환해 봅시다.
# -1은 '알아서 계산'을 의미합니다. (샘플 수, 784) 형태가 됩니다.
train_images_flat = train_images.reshape((train_images.shape[0], 28 * 28))
test_images_flat = test_images.reshape((test_images.shape[0], 28 * 28))

print(f"변환 후 학습 이미지 형태: {train_images_flat.shape}")
print(f"변환 후 테스트 이미지 형태: {test_images_flat.shape}")


# 3. 레이블 One-Hot Encoding: 정수 레이블을 10차원 벡터로 변환
# num_classes는 분류할 클래스 수 (0부터 9까지 10개)
num_classes = 10
train_labels_one_hot = to_categorical(train_labels, num_classes=num_classes)
test_labels_one_hot = to_categorical(test_labels, num_classes=num_classes)

print(f"원본 학습 레이블 (처음 5개): {train_labels[:5]}")
print(f"One-Hot 인코딩 후 학습 레이블 (처음 5개):")
print(train_labels_one_hot[:5])


## 3. 다중 클래스 분류를 위한 신경망 모델 구성

이제 Keras Sequential API를 사용하여 신경망 모델을 구성합니다. AND 게이트 예제와 비교하여 몇 가지 변경 사항이 있습니다.

*   **입력층 크기:** 28x28 이미지를 펼쳤으므로 입력 크기는 784가 됩니다.
*   **출력층 크기:** 0부터 9까지 10개의 클래스를 분류해야 하므로 출력 뉴런은 10개입니다.
*   **출력층 활성화 함수:** 각 클래스에 속할 **확률**을 계산하기 위해 **Softmax** 활성화 함수를 사용합니다. Softmax는 모든 출력 뉴런의 값을 0과 1 사이로 만들고, 이 값들의 총합이 1이 되도록 정규화합니다.
*   **은닉층 활성화 함수:** ReLU를 그대로 사용합니다.


In [ ]:
# 모델 구성
model = Sequential([
    # 입력층: 이미지를 1차원 벡터로 펼치는 레이어 (선택 사항, 위에서 미리 Flatten했으므로 주석 처리하거나 제거 가능)
    # Flatten(input_shape=(28, 28)), # 만약 이미지 형태 그대로 입력받는다면 Flatten 레이어 추가

    # 은닉층: 128개의 뉴런 (뉴런 수는 조절 가능), ReLU 활성화
    Dense(128, activation='relu', input_shape=(28 * 28,)), # 입력 크기 784 명시

    # # 과적합 방지를 위한 Dropout 레이어 추가 (나중에 실험해 봅시다)
    # Dropout(0.2), # 20%의 뉴런을 무작위로 비활성화

    # 출력층: 10개의 뉴런 (클래스 수), Softmax 활성화
    Dense(num_classes, activation='softmax')
])

# 모델 요약 정보 출력
model.summary()


모델 요약을 보면 파라미터 수가 상당히 늘어난 것을 볼 수 있습니다. (784 * 128 + 128) + (128 * 10 + 10) = 100480 + 1290 = 101770개입니다. AND 게이트 예제와 비교하면 훨씬 많습니다.

## 4. 모델 컴파일 (손실 함수 및 옵티마이저 정의)

모델을 학습시키기 전에 컴파일 단계를 거칩니다. 이때 손실 함수, 옵티마이저, 그리고 평가 지표를 정의합니다.

*   **손실 함수:** 다중 클래스 분류에는 **'categorical_crossentropy'** 또는 **'sparse_categorical_crossentropy'** 를 주로 사용합니다. 우리는 레이블을 원-핫 인코딩 했으므로 `'categorical_crossentropy'`를 사용합니다. (만약 레이블이 원본 정수 형태라면 `'sparse_categorical_crossentropy'`를 사용하면 편리합니다.) 크로스 엔트로피 손실은 모델의 예측 확률 분포와 실제 레이블(원-핫 벡터) 사이의 차이를 측정합니다. 차이가 클수록 손실이 커집니다.
*   **옵티마이저:** SGD 대신 **Adam** 옵티마이저를 사용해 봅시다. Adam은 학습률을 동적으로 조절하는 등 SGD보다 일반적으로 더 빠르고 안정적으로 학습하는 경향이 있어 널리 사용됩니다.
*   **평가 지표:** 분류 문제에서는 **'accuracy'** (정확도)를 가장 흔하게 사용합니다. 모델이 얼마나 많은 샘플을 올바르게 분류했는지 비율로 나타냅니다.


In [ ]:
# 모델 컴파일
# optimizer: 'adam' 사용 (Keras에서 문자열로 지정 가능)
# loss: 'categorical_crossentropy' 사용 (One-Hot 인코딩된 레이블에 적합)
# metrics: 학습 과정을 추적할 지표. 여기서는 'accuracy'를 사용합니다.
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("모델 컴파일 완료!")


## 5. 모델 학습 (미니 배치 학습)

이제 모델을 학습시킬 시간입니다. AND 게이트 예제에서는 전체 데이터를 한 번에 처리했지만, MNIST처럼 데이터셋 크기가 크면 전체 데이터를 한 번에 메모리에 올리거나 계산하기 어렵습니다. 또한, 미니 배치 학습은 학습 과정에 약간의 노이즈를 주어 더 넓은 최적해를 찾거나 학습 속도를 개선하는 데 도움이 됩니다.

Keras의 `model.fit()` 함수는 `batch_size` 매개변수를 통해 미니 배치 학습을 지원합니다. `batch_size`는 한 번의 가중치 업데이트에 사용할 데이터 샘플 수를 의미합니다.

`epochs`는 전체 데이터셋을 몇 번 반복하여 학습할지 지정합니다. `batch_size`와 `epochs`를 곱하면 전체 학습 반복 횟수가 됩니다. (정확히는 `(샘플 수 / batch_size) * epochs`)


In [ ]:
print("모델 학습 시작...")
# 학습 데이터(train_images_flat, train_labels_one_hot)와 학습 파라미터 지정
history = model.fit(train_images_flat, train_labels_one_hot,
                    epochs=10,        # 전체 데이터셋 10번 반복 학습
                    batch_size=32,    # 한 번에 32개 샘플씩 처리
                    verbose=1)        # 학습 과정 출력 상세도 (0: 없음, 1: 진행바, 2: 에포크당 한 줄)

print("모델 학습 완료!")


학습이 진행되면서 각 에포크마다 손실(loss)과 정확도(accuracy)가 출력되는 것을 확인할 수 있습니다. 손실은 줄어들고 정확도는 올라가는 것이 정상적인 학습 과정입니다.

`history` 객체에는 학습 과정 동안의 손실과 평가 지표 값들이 기록됩니다. 이를 이용해 학습 과정을 시각화해 볼 수 있습니다.


In [ ]:
# 학습 과정 시각화
plt.figure(figsize=(12, 4))

# 손실(Loss) 그래프
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.grid(True)

# 정확도(Accuracy) 그래프
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.grid(True)

plt.show()


그래프를 보면 에포크가 지남에 따라 손실은 감소하고 정확도는 증가하는 것을 확인할 수 있습니다.

## 6. 모델 평가

학습이 완료된 모델의 성능을 테스트 데이터셋으로 평가합니다. 학습에 사용되지 않은 데이터를 사용하여 모델이 새로운 데이터에 대해 얼마나 잘 작동하는지 확인하는 과정입니다.

`model.evaluate()` 함수를 사용하며, 반환 값은 컴파일 시 지정한 손실 값과 평가 지표 값들입니다.


In [ ]:
print("모델 평가 시작...")
# 테스트 데이터(test_images_flat, test_labels_one_hot)로 모델 평가
loss, accuracy = model.evaluate(test_images_flat, test_labels_one_hot, verbose=0) # verbose=0으로 하면 출력 최소화

print(f"테스트 손실(Loss): {loss:.4f}")
print(f"테스트 정확도(Accuracy): {accuracy:.4f}")
print("모델 평가 완료!")

In [1]:
5000_0000 * 100_000
5_000_000_000_000

5000000000000


테스트 정확도는 모델이 얼마나 일반화되어 새로운 데이터를 잘 처리하는지를 나타냅니다. 학습 정확도와 테스트 정확도를 비교하는 것이 중요합니다. 만약 학습 정확도는 매우 높은데 테스트 정확도가 significantly 낮다면, 모델이 학습 데이터에 **과적합(Overfitting)** 되었을 가능성이 높습니다.

## 7. 모델 예측

학습 및 평가가 끝났다면, 이제 모델을 이용해 실제 예측을 해볼 수 있습니다. `model.predict()` 함수를 사용합니다. 입력은 학습/테스트 데이터와 동일한 형태(batch_size, features)여야 합니다.


In [ ]:
# 테스트 데이터셋의 처음 5개 이미지에 대해 예측
predictions = model.predict(test_images_flat[:5])

print("처음 5개 테스트 이미지 예측 결과:")
print(predictions) # 각 샘플에 대해 10개 클래스에 대한 확률을 출력합니다.


`predict` 결과는 10개 클래스에 대한 확률 분포입니다. 예를 들어 첫 번째 샘플의 예측 결과가 `[0.01, 0.00, ..., 0.95, ..., 0.00]` 와 같다면, 이 모델은 이 샘플이 7번 클래스일 확률이 95%라고 예측한 것입니다.

우리가 원하는 최종 예측값은 가장 높은 확률을 가지는 클래스의 인덱스입니다. NumPy의 `argmax` 함수를 사용하여 이를 쉽게 얻을 수 있습니다.


In [ ]:
# 예측 결과 (확률 분포)를 실제 예측 클래스 (가장 확률 높은 클래스 인덱스)로 변환
predicted_classes = np.argmax(predictions, axis=1)

print(f"\n예측된 클래스 (처음 5개): {predicted_classes}")
print(f"실제 레이블 (처음 5개): {test_labels[:5]}") # 원본 정수 레이블과 비교


실제 레이블과 예측된 클래스를 비교하여 모델이 올바르게 예측했는지 확인할 수 있습니다.

몇 개의 샘플을 이미지와 함께 시각화하여 예측 결과를 확인해 봅시다.


In [ ]:
plt.figure(figsize=(12, 5))
for i in range(5):
    plt.subplot(1, 5, i + 1)
    # 원본 이미지 형태 (28, 28)로 되돌려서 시각화
    plt.imshow(test_images[i], cmap='gray')
    plt.title(f"Pred: {predicted_classes[i]}\nTrue: {test_labels[i]}")
    plt.axis('off')
plt.tight_layout()
plt.show()


## 8. 과적합 방지를 위한 Dropout 소개

MNIST 데이터셋은 비교적 단순하여 위 모델로도 꽤 높은 정확도를 얻을 수 있습니다. 하지만 더 복잡한 데이터셋이나 더 큰 모델을 사용할 경우, 모델이 학습 데이터에만 너무 맞춰져서 새로운 데이터에는 성능이 떨어지는 과적합(Overfitting) 문제가 발생하기 쉽습니다.

**Dropout**은 이러한 과적합을 방지하는 가장 효과적인 기법 중 하나입니다. 학습 과정 중 각 에포크마다 네트워크의 일부 뉴런을 무작위로 비활성화(출력을 0으로 만듦)하는 방식입니다. 마치 매번 다른 모델을 학습시키는 효과를 내어, 특정 뉴런에 의존하는 것을 방지하고 모델을 더 강건하게 만듭니다.

Keras에서는 `keras.layers.Dropout` 레이어를 추가하는 것만으로 쉽게 Dropout을 적용할 수 있습니다. Dropout 레이어는 일반적으로 활성화 함수를 거친 Dense 레이어 뒤에 추가됩니다. 비율(rate)은 비활성화할 뉴런의 비율을 의미합니다 (예: 0.2는 20% 비활성화).


In [ ]:
print("\n--- Dropout을 추가한 모델 구성 ---")

model_with_dropout = Sequential([
    # 입력층 (Flattening은 데이터 전처리 단계에서 했으므로 생략하거나 Flatten 레이어 추가)
    Dense(128, activation='relu', input_shape=(28 * 28,)),

    # Dropout 레이어 추가: 20%의 뉴런을 무작위로 비활성화
    # 학습 중에만 활성화되고, 예측/평가 시에는 비활성화되지 않습니다.
    Dropout(0.2),

    # 두 번째 은닉층 (선택 사항, 모델을 더 깊게 만들 수 있습니다)
    # Dense(64, activation='relu'),
    # Dropout(0.2),

    # 출력층
    Dense(num_classes, activation='softmax')
])

model_with_dropout.summary()

# Dropout이 추가된 모델 컴파일 및 학습
model_with_dropout.compile(optimizer='adam',
                           loss='categorical_crossentropy',
                           metrics=['accuracy'])

print("Dropout 포함 모델 학습 시작...")
# 학습 데이터를 사용하여 모델 학습 (epochs를 늘려볼 수도 있습니다)
history_dropout = model_with_dropout.fit(train_images_flat, train_labels_one_hot,
                                          epochs=10, # 동일한 epoch 수로 비교
                                          batch_size=32,
                                          verbose=1)

print("Dropout 포함 모델 학습 완료!")

# Dropout 포함 모델 평가
loss_dropout, accuracy_dropout = model_with_dropout.evaluate(test_images_flat, test_labels_one_hot, verbose=0)

print(f"Dropout 포함 모델 테스트 손실(Loss): {loss_dropout:.4f}")
print(f"Dropout 포함 모델 테스트 정확도(Accuracy): {accuracy_dropout:.4f}")

# 학습 과정 시각화 (Dropout 포함 모델)
plt.figure(figsize=(12, 4))

# 손실(Loss) 그래프
plt.subplot(1, 2, 1)
plt.plot(history_dropout.history['loss'])
plt.title('Model Loss (with Dropout)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.grid(True)

# 정확도(Accuracy) 그래프
plt.subplot(1, 2, 2)
plt.plot(history_dropout.history['accuracy'])
plt.title('Model Accuracy (with Dropout)')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.grid(True)

plt.show()


Dropout을 추가하면 학습 데이터에 대한 정확도가 약간 떨어지거나 학습 속도가 느려지는 것처럼 보일 수 있지만, 테스트 데이터에 대한 정확도는 향상되거나 과적합이 줄어드는 효과를 기대할 수 있습니다. MNIST는 과적합이 심하게 발생하지 않는 간단한 데이터셋이라 큰 차이가 없을 수도 있습니다. 하지만 더 복잡한 문제에서는 Dropout의 효과가 두드러집니다.

## 9. 마치며

축하합니다! 이제 AND 게이트를 넘어 MNIST와 같은 좀 더 현실적인 데이터셋을 다루고, 다중 클래스 분류 문제 해결을 위한 핵심 기법들(Softmax, Cross-Entropy, Adam, 미니 배치 학습, Dropout)을 익혔습니다.

이 단계에서 더 나아가 볼 수 있는 것들은 다음과 같습니다.

*   **하이퍼파라미터 튜닝:** 은닉층 뉴런 수, 학습률, 배치 크기, 에포크 수, Dropout 비율 등을 바꿔가며 모델 성능을 최적화하는 실험을 해보세요.
*   **모델 구조 변경:** 은닉층을 더 추가하여 네트워크를 깊게 만들어 보세요. (심층 신경망 - Deep Neural Network)
*   **다른 데이터셋 적용:** Fashion MNIST, CIFAR-10 등 다른 이미지 분류 데이터셋에 동일한 과정을 적용해 보세요. (CIFAR-10은 MNIST보다 훨씬 어렵습니다!)
*   **합성곱 신경망 (Convolutional Neural Network, CNN):** 이미지 데이터에 특화된 CNN은 Dense 레이어만 사용하는 현재 모델보다 훨씬 뛰어난 성능을 보여줍니다. 다음 단계로 CNN을 학습하는 것을 강력 추천합니다!
*   **순환 신경망 (Recurrent Neural Network, RNN):** 텍스트, 시계열 데이터 등 순차적인 데이터를 다루는 데 사용되는 신경망입니다.
*   **Keras Functional API 또는 PyTorch 활용:** Keras Sequential API는 간단한 모델에 좋지만, 더 복잡하거나 다중 입력/출력을 가진 모델을 만들 때는 Functional API가 유용합니다. PyTorch로도 동일한 과정을 구현해 볼 수도 있습니다.
